In [1]:
!pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from dotenv import load_dotenv, find_dotenv
from langchain_community.retrievers import PineconeHybridSearchRetriever
load_dotenv(find_dotenv())
import os
api_key = os.environ['PINECONE_API_KEY']
#print(api_key)

In [4]:
from pinecone import Pinecone, ServerlessSpec
index_name ="hybrid-search-langchain-pinecone"
#initialize pinecone client
pc = Pinecone(
    api_key=api_key,
)
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, # dimesnion of dense vector
        metric="dotproduct", ## sparse values supported only fo dotproduct
        spec = ServerlessSpec(cloud='aws',region='us-east-1')
        
    )

In [5]:
index = pc.Index(index_name)
index 

In [7]:
!pip install langchain_huggingface


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
  Using cached tokenizers-0.19.1-cp312-none-win_amd64.whl.metadata (6.9 kB)
  Using cached transformers-4.42.4-py3-none-any.whl.metadata (43 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached torch-2.3.1-cp312-cp312-win_amd64.whl.metadata (26 kB)
  Using cached scikit_learn-1.5.1-cp312-cp312-win_amd64.whl.metadata (12 kB)
  Using cached scipy-1.14.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pillow-10.4.0-cp312-cp312-win_amd64.whl.metadata (9.3 kB)
  Using cached safetensors-0.4.3-cp312-none-win_amd64.whl.metadata (3.9 kB)
  Using cached sympy-1.13.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached thre

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
embeddings

In [9]:
from pinecone_text.sparse import BM25Encoder
bm25_encoder = BM25Encoder().default()
bm25_encoder

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Darshan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [10]:
sentences = [
    "In 2023, I visited London",
    "In 2022, I visited India",
    "In 2021, I visited Connecticut"
]

bm25_encoder.fit(sentences)

#store the values to json file
bm25_encoder.dump("bm25_encoder.json")

#load the BM25 encoder from json file
bm25_encoder = BM25Encoder().load("bm25_encoder.json")

100%|██████████| 3/3 [00:00<00:00, 11.35it/s]


In [11]:
retriever = PineconeHybridSearchRetriever(
    embeddings=embeddings,
    sparse_encoder= bm25_encoder,
    index= index
)
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000001B282B0F3B0>, index=<pinecone.data.index.Index object at 0x000001B2E3D14C20>)

In [12]:
retriever.add_texts(
    [
    "In 2023, I visited London",
    "In 2022, I visited India",
    "In 2021, I visited Connecticut"
]
)

100%|██████████| 1/1 [00:10<00:00, 10.82s/it]


In [13]:
retriever.invoke("what city did i visit last?")

[Document(page_content='In 2021, I visited Connecticut'),
 Document(page_content='In 2023, I visited London'),
 Document(page_content='In 2022, I visited India')]